# FRI: Fast Reed Solomon IOP of Proximity

---
## FRI Low Degree Tests
---

In [15]:
%load_ext autoreload
%autoreload 2

from src.proth_primes import proth_in_range
from src.fri import FRIOPP, FRIPCS_Prover, FRIPCS_Verifier
from sage.rings.finite_rings.all import GF
from sage.rings.polynomial.all import PolynomialRing
from sage.misc.functional import log
from sage.misc.prandom import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
(p, k, n) = proth_in_range(15, 15, 27,27)
print(f"Using prime {p} = {k}⋅2^{n} + 1")
Fq = GF(p)
omega = Fq.multiplicative_generator() 
omega = omega**k
omega_2sylow_expo = log(omega.multiplicative_order(), 2)
print(f"Maximal 2-sylow multiplicative generator: {omega}")
print(f"Maximal 2-sylow multiplicative power: {omega_2sylow_expo}")

Using prime 2013265921 = 15⋅2^27 + 1
Maximal 2-sylow multiplicative generator: 440564289
Maximal 2-sylow multiplicative power: 27


### Setup

In [17]:
polynomial_degree = 512
folding_factor = 4
rate_factor = 4
rs_code_length = rate_factor*polynomial_degree
rs_code_log_len = int(log(rs_code_length, 2))
PolyRing = PolynomialRing(Fq, "X")
ntt_omega = omega**(2**(omega_2sylow_expo - rs_code_log_len))
assert ntt_omega.multiplicative_order() == rs_code_length

input_polynomial = PolyRing.random_element(polynomial_degree-1)

print(f"Using polynomial: {input_polynomial}")

Using polynomial: 763501829*X^511 + 1807954389*X^510 + 1010159676*X^509 + 800793533*X^508 + 572367765*X^507 + 1093280211*X^506 + 1545744816*X^505 + 236139188*X^504 + 1055747734*X^503 + 1447685872*X^502 + 1303535268*X^501 + 706097876*X^500 + 127914497*X^499 + 1940461334*X^498 + 1576301888*X^497 + 1636785102*X^496 + 906488656*X^495 + 1643105980*X^494 + 47928387*X^493 + 698160244*X^492 + 165538187*X^491 + 1998189688*X^490 + 617795010*X^489 + 1032840023*X^488 + 1197963713*X^487 + 1551897010*X^486 + 1474850138*X^485 + 266830523*X^484 + 1824056998*X^483 + 418029020*X^482 + 550263385*X^481 + 1268286676*X^480 + 1470619780*X^479 + 1996212764*X^478 + 724939666*X^477 + 172362752*X^476 + 1518568072*X^475 + 1946592489*X^474 + 220001987*X^473 + 1907439836*X^472 + 866822822*X^471 + 335458398*X^470 + 1599645906*X^469 + 1456265698*X^468 + 1981740990*X^467 + 1324618632*X^466 + 1615109886*X^465 + 1103527888*X^464 + 135595232*X^463 + 311321311*X^462 + 1833470208*X^461 + 74971974*X^460 + 1349958225*X^459 +

### Low degree test

In [18]:
iopp = FRIOPP(input_polynomial=input_polynomial, 
              ntt_omega=ntt_omega, 
              rate_factor=rate_factor, 
              folding_factor=folding_factor)

max_degree = polynomial_degree

if iopp.verify_proximity(max_degree=max_degree) is True:
    print("✓ Positive case of low degree test passed")
else:
    print("❌ERROR: Positive case of low dergee test failed!")

if iopp.verify_proximity(max_degree // folding_factor  ) is False:
    print("✓ Negative case case of low degree test passed")
else:
    print("❌ ERROR: Negative case of low dergee test failed!")


✓ Positive case of low degree test passed
✓ Negative case case of low degree test passed


---
## FRI Polynomial Commitment Scheme
---

In [19]:
evaluation_point = PolyRing.base_ring().random_element()

prover = FRIPCS_Prover(ntt_omega, rate_factor, folding_factor)
commitment = prover.commit(input_poly=input_polynomial)
(expected_value, proof) = prover.open(evaluation_point)

verifier = FRIPCS_Verifier(polynomial_degree - 1)

if verifier.verify_proof(commitment, proof, evaluation_point, expected_value) == True:
        print(f"✓ A valid proof opening succeeded")
else:
        print(f"❌A valid opening should succeed, but it didn't")

if verifier.verify_proof(commitment, proof, evaluation_point + 1, expected_value) == False:
        print("✓ An invalid proof opening failed as expected")
else:
        print(f"❌Invalid opening of an evaluation point should not succeed, but it did!")

if verifier.verify_proof(commitment, proof, evaluation_point, expected_value - 1) == False:
        print("✓ Another invalid proof opening failed as expected")
else:
        print(f"❌Invalid opening of an evaluation point should not succeed, but it did!")

✓ A valid proof opening succeeded
✓ An invalid proof opening failed as expected
✓ Another invalid proof opening failed as expected
